# Comdirect Webscraping

In diesem Notebook werden die Bauzinsen von Comdirect gescrapet. Die Daten werden u.s. für eine Korrelationsanalyse mit Häuserpreisen verwendet.

## Inhaltsverzeichnis

1. [Installationen](#1-installationen)
2. [Scrapen der Werte](#2-scrapen-der-werte)
3. [Umwandlung in Dataframe](#3-umwandlung-in-dataframe)

## 1. Installationen

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
from lxml import html

## 2. Scrapen der Werte

Die Bauzinsen sind in einer Tabelle auf einer Webseite angeben. Es muss daher nicht auf verschiedene Seiten gegangen werden, sondern eine einzige URL wird verwendet. 

Wie auch beim Scrapen von Immowelt, wird XPath verwendet. Es werden die effektiven Jahreszinsen für eine Sollzinsbindung von 5, 10, 15, 20 und 25 Jahren angezogen. Außerdem wird das aktuelle Datum mit Uhrzeit gescrapet, sodass später die Veränderung im Laufe der Zeit betrachtet werden kann.

In [2]:
url = "https://www.comdirect.de/kredit/bauzinsen.html#Bauzinsen"

page = requests.get(url)
tree = html.fromstring(page.content)

eff5 = tree.xpath('//div[@class="table__container--scroll"]/table/tbody/tr[1]/td[4]/eff5/text()')
eff10 = tree.xpath('//div[@class="table__container--scroll"]/table/tbody/tr[2]/td[4]/eff10/text()')
eff15 = tree.xpath('//div[@class="table__container--scroll"]/table/tbody/tr[3]/td[4]/eff15/text()')
eff20 = tree.xpath('//div[@class="table__container--scroll"]/table/tbody/tr[4]/td[4]/eff20/text()')
eff25 = tree.xpath('//div[@class="table__container--scroll"]/table/tbody/tr[5]/td[4]/eff25/text()')

stand = tree.xpath('//div[@class="col__content outer-spacing--xlarge-bottom"]/p/date/text()')

print(eff5)
print(eff10)
print(eff15)
print(eff20)
print(eff25)
print(stand)

['3,40']
['3,39']
['3,43']
['3,66']
['3,90']
['21.12.2022 16:30']


## 3. Umwandlung in Dataframe

Die erhaltenen Werte werden nun in ein Pandas Dataframe umgewandelt, damit sie besser zu kombinieren und weiterverarbeiten sind. Zuerst wird eine Matrix erstellt, welche in der ersten Spalte die Dauer der Sollzinsbindung angibt, in der Zweiten den effektiven Jahreszins (gescrapte eff* Werte) und in der Dritten das Datum mit Uhrzeit.

In [14]:
matrix = [[0 for s in range(10)] for r in range(10)]

matrix[0][0]="Sollzinsbindung"
matrix[0][1]="5 Jahre"
matrix[0][2]="10 Jahre"
matrix[0][3]="15 Jahre"
matrix[0][4]="20 Jahre"
matrix[0][5]="25 Jahre"
matrix[1][0]="Effektiver Jahreszins"
matrix[1][1]=eff5
matrix[1][2]=eff10
matrix[1][3]=eff15
matrix[1][4]=eff20
matrix[1][5]=eff25
matrix[2][0]="Datenstand"
matrix[2][1]=stand
matrix[2][2]=stand
matrix[2][3]=stand
matrix[2][4]=stand
matrix[2][5]=stand



Die Umwandlung der Matrix in ein Dataframe wird wie beim Scraping von Immowelt umgesetzt.

In [15]:
data = {
    'Sollzinsbindung': [],
    'Effektiver Jahreszins': [],
    'Datenstand': [],
}

for i in range(5):
    data['Sollzinsbindung'].append(matrix[0][i+1])
    data['Effektiver Jahreszins'].append(matrix[1][i+1])
    data['Datenstand'].append(matrix[2][i+1])

df=pd.DataFrame(data, columns=['Sollzinsbindung','Effektiver Jahreszins','Datenstand'])

df.to_csv('Zinsen.csv')

Zuletzt wird überprüft, ob die Umwandlung korrekt funktioniert hat, indem das Dataframe angezeigt wird.

In [16]:
df

,Sollzinsbindung,Effektiver Jahreszins,Datenstand
0,5 Jahre,"[3,40]",[21.12.2022 16:30]
1,10 Jahre,"[3,39]",[21.12.2022 16:30]
2,15 Jahre,"[3,43]",[21.12.2022 16:30]
3,20 Jahre,"[3,66]",[21.12.2022 16:30]
4,25 Jahre,"[3,90]",[21.12.2022 16:30]


Die Umwandlung hat korrekt funktioniert. Dieser Code wird nun als Grundlage für das tägliche Webscraping verwendet.